In [68]:
#importメソッド
import spacy   #spacyライブラリ（自然言語処理のため）
import ginza   #ginzaライブラリ（spacyを日本語対応させる）
from spacy import displacy   #displacyライブラリ（文章構造の視覚化）
import csv     #csvライブラリ（TSVファイルの入出力に必要）

#定数宣言
SUBJECT = "Who"#主語のproperty分類を表した定数
OBJ = "Obj"#目的語のproperty分類を表した定数
#FILE_NAME = 'プログラム試験用'
#FILE_NAME = 'DevilsFoot'
#FILE_NAME = 'SilverBlaze'
#FILE_NAME = 'ResidentPatient'
#FILE_NAME = 'DancingMen置換'#元データからプログラム処理の邪魔になる"を置換したデータ
#FILE_NAME = 'ACaseOfIdentity'
FILE_NAME = 'AbbeyGrange'

In [69]:
nlp = spacy.load('ja_ginza_electra')


class SentSVO:                #文章を主語・述語・目的語に分解するクラス
    def __init__(self,sent,sent_id):
        self.subject = [""]        #主語となる文字列を保存するlist型変数(主語が複数ある場合は要素ごとに分ける
        self.obj = []              #目的語となる文字列を保存するlist型変数(主語が複数ある場合は要素ごとに分ける
        self.obj_list_num = 0         #目的語の要素数（目的語はsent.rootの子トークンに複数あるためインタスタンス変数として管理）
        self.hasPredicate = ""   #述語となる文字列を保存するlist型変数(主語が複数ある場合は要素ごとに分ける
        self.first_num = 0          #係り受け関係（自身含む）にある最初の単語番号を探すための変数(初期値は変数として宣言するためのもの)
        self.last_num = 0           #係り受け関係（自身含む）にある最後の単語番号を探すための変数(初期値は変数として宣言するためのもの)
        self.sent = sent            #解析する文
        self.sent_id = sent_id      #文章ID
        self.cheak = True       #SVO分解したものの正誤判定のための変数
        self.scene_cheak = ["〇","〇","〇"]  #場面ごとのSVOの正誤判定(リストの0:主語、1:述語、2:目的語)
        self.dislocated_flag = False  #転置「dislocated」の有無を判定する変数
        self.property_stock = ""  #目的語・述語のプロパティ保存用（インスタンス変数にしてあるのはプロパティの分類が出来るようになったときに正誤判定部分を関数分けするため）
    
    def first_search(self,children):          #係り受け（自身含む）の最初を探して、単語番号を保存するクラス関数
        if not children:                      #子トークンがない場合は関数終了
            return
        elif self.first_num > children[0].i:         
            self.first_num = children[0].i
            self.first_search(list(children[0].children))
    
    def last_search(self,children):       #係り受け（自身含む）の最後を探して、単語番号を保存するクラス関数
        if not children:                  #子トークンがない場合は関数終了
            return
        elif self.last_num < children[-1].i:
            self.last_num = children[-1].i
            self.last_search(list(children[-1].children))

    def subject_search(self,token):    #この単語に係り受けしている単語の中から主語として必要なものを付け足すクラス関数
        self.first_num = token.i               #自身の単語番号で初期化（最初の単語番号）
        self.last_num = token.i                #自身の単語番号で初期化（最後の単語番号）
        children = list(token.children)    #単語tokenを係り受けしている単語（複数）を取得し、list型でchildrenに代入
        self.first_search(children)          #主語の最初の単語番号を探す
        self.last_search(children)           #主語の最後の単語番号を探す
        list_num = 0                           #リストsubjectの要素数を管理するための変数


        if self.sent[self.last_num].dep_ == "punct":  #句読点だった場合は一つ前の単語で判定
            self.last_num -= 1
            
        if not self.sent[self.last_num].dep_ == "case":       #係り受け関係の最後の単語が助詞でない場合はその単語も含む
            self.last_num += 1
        
        for num in range(self.first_num,self.last_num):     #係り受け関係の最初の単語から最後の単語まで順番に付け足していく
            if self.sent[num].dep_=="case" and (self.sent[num].text=="と" or self.sent[num].text=="や"):
                #特定の助詞の場合は付け足さずに要素数を増やし以後そちらに単語を付け足す
                list_num += 1
                self.subject.append("")
                continue
            self.subject[list_num] += self.sent[num].text   #単語を結合していく
        
    def obj_search(self,token):              #この単語に係り受けしている単語の中から目的語として必要なものを付け足すクラス関数
        self.first_num=token.i               #自身の単語番号で初期化（最初の単語番号）
        self.last_num=token.i                #自身の単語番号で初期化（最後の単語番号）
        children=list(token.children)      #単語tokenを係り受けしている単語（複数）を取得し、list型でchildrenに代入
        self.first_search(children)          #目的語の最初の単語番号を探す
        self.last_search(children)           #目的語の最後の単語番号を探す
        
        #↓係り受け関係の最後の単語が助詞でない場合はその単語も含む
        if not self.sent[self.last_num].dep_ == "case":  
            self.last_num += 1
 
        for num in range(self.first_num,self.last_num):     #係り受け関係の最初の単語から最後の単語まで順番に付け足していく
            if self.sent[num].dep_=="case" and (self.sent[num].text=="と" or self.sent[num].text=="や" \
            or self.sent[num].text=="から"):#特定の助詞の場合は付け足さずに要素数を増やして以後そちらに単語を付け足す
                self.obj_list_num += 1
                self.obj.append("")
                continue
            self.obj[self.obj_list_num] += self.sent[num].text  #単語を結合していく
            #print("残ってる",self.obj)
            
    def hasPredicate_search(self,token):       #述語の取得
        self.first_num = token.i               #自身の単語番号で初期化（最初の単語番号）
        self.last_num = token.i                #自身の単語番号で初期化（最後の単語番号）
        rootchildren=list(token.children)  #単語tokenを係り受けしている単語（複数）を取得し、list型でchildrenに代入
        
        #述語はrootスタートなので一度目の単語番号索敵は条件付けが必要
        for rootchild in rootchildren:   #主語系、目的語系以外の係り受け先で最も単語番号の小さい単語を探索
            if self.first_num > rootchild.i:
                if self.dislocated_flag:  #転置がある場合は"obl""obj""dislocated"以外から
                    if "obl"!=rootchild.dep_ and "obj"!=rootchild.dep_ and "dislocated"!=rootchild.dep_:
                        self.first_num = rootchild.i
                        self.first_search(list(rootchild.children))    #述語の最初の単語番号を探す
                else:  #転置がない場合は"obl""obj""nsubj""csubj"以外から
                    if "nsubj"!=rootchild.dep_ and "csubj"!=rootchild.dep \
                    and "obl"!=rootchild.dep_ and "obj"!=rootchild.dep_ :
                        self.first_num = rootchild.i
                        self.first_search(list(rootchild.children))    #述語の最初の単語番号を探す                        
            else:
                break
                
        for rootchild in reversed(rootchildren):#主語系、目的語系以外の係り受け先で最も単語番号の大きい単語を探索
            if self.last_num < rootchild.i:
                if self.dislocated_flag:  #転置がある場合は"obl""obj""dislocated"以外から
                    if "obl"!=rootchild.dep_ and "obj"!=rootchild.dep_ and "dislocated"!=rootchild.dep_:
                        self.last_num = rootchild.i
                        self.last_search(list(rootchild.children))    #述語の最後の単語番号を探す                          
                else:  #転置がない場合は"obl""obj""nsubj""csubj"以外から
                    if "nsubj"!=rootchild.dep_ and "csubj"!=rootchild.dep \
                    and "obl"!=rootchild.dep_ and "obj"!=rootchild.dep_ :
                        self.last_num = rootchild.i
                        self.last_search(list(rootchild.children))    #述語の最後の単語番号を探す
            else:
                break
          
        #係り受け関係の最後の単語が助詞でない場合はその単語も含む
        if not self.sent[self.last_num].dep_ == "case":  
            self.last_num += 1
        
        #係り受け関係の最初の単語から最後の単語まで順番に付け足していく
        for num in range(self.first_num,self.last_num):
            self.hasPredicate += self.sent[num].text  #単語を結合していく
            #print("残ってる",self.obj)
        
    def svo_parse(self):   #主語・述語・目的語の分解を行うクラス関数（外部から呼び出す）        
        if not self.sent:   #文字列が存在しない場合はreturn
            return
        
        for token in self.sent:
            if token.dep_ == "dislocated":   #解析した文から転置「dislocated」を探す
                self.dislocated_flag = True
                self.subject_search(token) #転置先を主語として扱う
            
        rootchildren = list(self.sent.root.children)   #単語root（述語）に直接係り受けしている単語をlist型で取得
        for rootchild in rootchildren:                              #取得した単語（複数）をすべて参照
            if "nsubj"==rootchild.dep_ or "csubj"==rootchild.dep_:  #主語や主部の単語があれば
                if not self.dislocated_flag:                        #転置が無ければ
                    self.subject_search(rootchild)                #主語（主部）の係り受け関係を取得
            elif "obl"==rootchild.dep_ or "obj"==rootchild.dep_:    #目的語の単語があれば
                self.obj.append("")                                   #目的語を保存するlistの要素を追加する
                self.obj_search(rootchild)                            #目的語の係り受け関係をすべて取得
                self.obj_list_num += 1                                  #次に参照する目的語のlistの要素を変更
        self.hasPredicate_search(self.sent.root)                  #述語の係り受け先を取得
        
    
    def svo_cheak(self,token_mode,cheak_list,token,scene_cheak_num):        #取得結果と手本（人の手で作ったもの）の比較のためのクラス関数
            self.cheak = False   #正誤判定の変数
            #print(cheak_list)#テスト用
            for cheak in cheak_list:  #この場面IDにおける答えの単語群を一行ずつループ参照
                #print(cheak[1],token_mode,"1つめ")#テスト用
                if cheak[1] == token_mode:
                    #print(cheak[2],token,"2つめ")#テスト用
                    if cheak[2] == token:    #SVO分解して取得した結果と手本の答えを比較
                        self.cheak = True       #cheak_listにあったことを記録
                        break
    
    def svo_print(self,cheak_list,true_properties_num,token_properties):           #主語・述語・目的語を出力する関数
        with open("../experiment/"+FILE_NAME+"SVO結果.tsv",mode="a+",encoding='utf-8',newline='') as output_file:   #出力ファイルを作成し、一行ずつ出力して追加する
            writer = csv.writer(output_file,delimiter="\t")   #出力形式をTSVファイルに設定
            #print(self,cheak_list)#正解データの確認テスト用
            
            #対象文
            line_stock = [[self.sent_id,"対象文",self.sent,"",""]]  #対象文行をリストに保存
            #print(cheak_list)#テスト用
            
            #主語
            part_cheak = "×"   #場面ごとの判定において×か△を評価するための変数
            for subject in self.subject:                             
                self.svo_cheak(SUBJECT,cheak_list,subject,0)                       #主語を正誤判定
                if self.cheak:#正解であればpart_cheakを△にする
                    part_cheak = "△"
                else:#不正解であればself.scene_cheak[0]を×にする
                    self.scene_cheak[0] = "×"
                line_stock.append([self.sent_id,SUBJECT,subject,self.cheak,""])   #主語行をリストに保存
           
            if self.scene_cheak[0] == "×":#主語の中に不正解があるなら、part_cheak（全て不正解か部分不正解か）を反映
                self.scene_cheak[0] == part_cheak
            if len(self.subject) == 0:#主語が一つもない場合は"無"
                self.scene_cheak[0] = "無"
                
            #述語
            #self.svo_cheak("hasPredicate",cheak_list,self.hasPredicate)#述語を正誤判定（インスタンス内変数self.hasPredicateを引数にしているのは単語ごとに関数を使い分けるため）
            #####現状は述語の種類分け（hasPredicateかhasProperty）まで出来てないので述語の正誤判定は以下でする###############
            self.cheak = False   #正誤判定の変数（0なら正、1なら誤）
            for cheak in cheak_list:  #この場面IDにおける答えを一行ずつループ参照
                if cheak[1]=="hasProperty" or cheak[1]=="hasPredicate":
                    if cheak[2] == self.hasPredicate:    #SVO分解して取得した結果と手本の答えを比較
                        self.cheak = True       #cheak_listにあったことを記録
                        self.property_stock = cheak[1]   #述語のプロパティの種類を保存する
            ################################################################################################################                      
            if self.cheak:
                #self.property_stockと一致する述語のプロパティ数を加算（正解数）
                if self.property_stock==token_properties[16]:#正解のプロパティが"hasProperty"であれば
                    true_properties_num[16] += 1   #正解数を加算
                if self.property_stock==token_properties[17]:#正解のプロパティが"hasPredicate"であれば
                    true_properties_num[17] += 1   #正解数を加算
            else:#不正解であればself.scene_cheak[1]を×にする
                    self.scene_cheak[1] = "×"            
            line_stock.append([self.sent_id,"hasPredicate",self.hasPredicate,self.cheak,self.property_stock])    #述語行をリストに保存
            
            #目的語
            part_cheak = "×"   #場面ごとの判定において×か△を評価するための変数
            for obj in self.obj:            
                #####現状は目的語の種類分けまで出来てないので目的語の正誤判定は以下でする###############
                self.cheak = False   #正誤判定の変数（0なら正、1なら誤）
                self.property_stock = "判別不能" #答えにおける目的語のプロパティ
                for cheak in cheak_list:  #この場面IDにおける答えを一行ずつループ参照
                    if cheak[1]!=SUBJECT and cheak[1]!="hasPredicate" and cheak[1]!="hasProperty":
                        if cheak[2] == obj:    #SVO分解して取得した結果と手本の答えを比較
                            self.cheak = True       #cheak_listにあったことを記録
                            self.property_stock = cheak[1]   #目的語のプロパティの種類を保存する
                            break#SVO結果と一致する答えがあれば、ループを終了する。
                ########################################################################################  
                if self.cheak:  #self.cheakが真の場合（self.cheakにTrueを代入する時に行わないのはプロパティの判別が出来れば関数svo_cheakで行うから）
                    part_cheak = "△"   #正解であればpart_cheakを△にする
                    #↓self.property_stockと一致する目的語のプロパティ数を加算（正解数）
                    for num in range(16):            #目的語のプロパティ数分ループ
                        if self.property_stock==token_properties[num]:#正解のプロパティと目的語の各プロパティを比較
                            true_properties_num[num] += 1
                else:#不正解であればself.scene_cheak[2]を空にする
                    self.scene_cheak[2] = "×"
                line_stock.append([self.sent_id,OBJ,obj,self.cheak,self.property_stock]) #SVO結果の目的語を行としてリストに保存               
            
            if self.scene_cheak[2] == "×":#主語の中に不正解があるなら、part_cheak（×か△）を反映
                self.scene_cheak[2] = part_cheak
            if self.obj_list_num == 0:#目的語が一つもない場合は"無"
                self.scene_cheak[2] = "無"
            #print(part_cheak,self.scene_cheak[2],self.sent_id) #テスト用
            
            for line in line_stock:#場面の各行を纏めて出力
                writer.writerow(line+self.scene_cheak)
        
        return true_properties_num

#tsvファイルの読み込み
sent_id = "0"  #場面ID（文ごとのIDでもある）

all_properties_num = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]#目的語の各プロパティの総数(詳しくは「現状とプロパティ」より)
true_properties_num = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]#目的語の各プロパティの正解数(詳しくは「現状とプロパティ」より)
token_properties = ["Why","What","How","When",\
                  "Where","Whom","From","To",\
                  "On","Under","Left","Right",\
                  "Near","NextTo","MiddleOf","Opposite",\
                  "hasProperty","hasPredicate"]
"""
↑目的語と述語の各プロパティの種類
(変数「all_properties_num」と変数「true_properties_num」の要素順と対応している)
"""

with open("../experiment/"+FILE_NAME+"SVO結果.tsv",mode="w",encoding='utf-8',newline='') as file:
    print("出力ファイルの初期化") #出力ファイルの初期化       
    writer = csv.writer(file,delimiter="\t")   #出力形式をTSVファイルに設定
    writer.writerow(["ID","Property","原文","正誤判定","正解Propertyの詳細","場面内の主語","場面内の述語","場面内の目的語"])#見出し行（エクセルにしたとき）
with open("../experiment/"+FILE_NAME+".tsv",mode="r", encoding='utf-8', newline='') as input_file:      #入力ファイルを読み込む
    read_list = list(csv.reader(input_file, delimiter='\t'))    #読み込んだ入力ファイルをtsv形式に整形してリスト型として代入
    for cols in read_list:
        #print(cols)   #上手くいっていない行確認テスト用
        if "対象文" == cols[1]:                        #2列目propertyが「対象文」である行を取得
            sent_id = cols[0]                          #場面IDを更新
            doc = nlp(cols[2])                         #取得した行の3列目原文を取得
            #print(list(doc.sents))#if len(list(doc.sents))==1:のテスト用
            if len(list(doc.sents))==1:
                for sent in doc.sents: #文章を一文ずつ分けてループ処理する(将来短文ではなく長文に対処するため)
                    #print(sent)  #上手くいっていない対象文確認テスト用
                    svo = SentSVO(sent,sent_id)          #インスタンスの作成
                    svo.svo_parse()                      #SVO分解を行う

                    #グラフ表示（試験用）
                    #displacy.render(doc, style='dep', jupyter=True, options={'compact':True, 'distance': 90})

                    index_num = read_list.index(cols)   #read_listの現在参照している要素数
                    cheak_list = []   #現在の場面IDにおける入力ファイルの正解を保存するためのリスト型変数

                    #read_listの（colsの要素数＋１）から（最後の要素数）までループ<same_num_colsにはcolsと同じ場面IDである１行が格納される>
                    for same_num_cols in read_list[index_num+1:read_list.index(read_list[-1])+1]:
                        if sent_id!=same_num_cols[0]:    #"対象文"と現在の行が場面IDが異なる場合はループを終了
                            break
                        else:
                            if not same_num_cols[2]:#行の要素2（主語とか述語とかが入っている）が空ならスキップ
                                continue
                            else:
                                #print(read_list[index_num+addition_num][2])#テスト用
                                cheak_list.append(same_num_cols)
                                #答えのリストから目的語の種類と一致する場合は加算
                                for num in range(18):
                                    if same_num_cols[1]==token_properties[num]:#same_num_colsを述語・目的語のプロパティと同じものがないか比較（主語はプロパティが一つしかないので判定しない）
                                        all_properties_num[num] += 1#同じものがあれば、そのプロパティの総数を加算
                    #SVO分解結果を出力し、目的語プロパティの種類ごとに正解数を加算してtrue_properties_numに保存。
                    true_properties_num = svo.svo_print(cheak_list,true_properties_num,token_properties)
            else:
                #print("2文以上あるので回避します",list(doc.sents))
                with open("../experiment/"+FILE_NAME+"SVO結果.tsv",mode="a+",encoding='utf-8',newline='') as output_file:   #出力ファイルを作成し、一行ずつ出力して追加する
                    writer = csv.writer(output_file,delimiter="\t")   #出力形式をTSVファイルに設定                
                    writer.writerow([sent_id,"対象文",cols[2],"","","","","","解析不可"])

#記録した目的語プロパティの種類ごとの正解数と総数をファイルに出力
with open("../experiment/"+FILE_NAME+"プロパティ.tsv",mode="w",encoding='utf-8',newline='') as obj_file:  
    obj_writer = csv.writer(obj_file,delimiter="\t")   #出力形式をTSVファイルに設定              
    for property_num in range(18):
        obj_writer.writerow([token_properties[property_num],true_properties_num[property_num],all_properties_num[property_num]])
print("出力完了")#プログラムが正常に終了した知らせ

出力ファイルの初期化
出力完了


In [70]:
#FILE_NAME = '実験実験'#場面の正解数を正しく数えられているかを評価するテスト用のファイル
#SVO結果を評価するプログラム
all_subject = 0 #すべての主語の数
true_subject = 0 #正解の主語の数
all_hasPredicate = 0 #すべての述語の数
true_hasPredicate = 0 #正解の述語の数
all_obj = 0 #すべての目的語の数
true_obj = 0 #正解の目的語の数
true_scene = 0 #全て正解だった場面数

with open("../experiment/"+FILE_NAME+"SVO結果.tsv",mode="r",encoding='utf-8') as input_file:    #「SVO結果」を開く
        read_list = list(csv.reader(input_file, delimiter='\t'))#開いたファイルをtsv形式で読み込む
        all_scene = int(read_list[-1][0]) #全場面数
        for cols in read_list:  #読み込んだリストを一行ずつループ処理
            if SUBJECT==cols[1]:      #主語を探索
                all_subject += 1
                if "True"==cols[3]:            #正解しているかどうか
                    true_subject += 1
            elif "hasPredicate"==cols[1] or "hasProperty" == cols[1]: #述語を探索
                all_hasPredicate += 1
                if "True"==cols[3]:            #正解しているかどうか
                    true_hasPredicate += 1
            elif OBJ==cols[1]:          #目的語を探索
                all_obj += 1
                if "True"==cols[3]:            #正解しているかどうか
                    true_obj += 1
            if cols[1]=="対象文":
                if cols[5]=="〇" and cols[6]=="〇" and (cols[7]=="〇" or cols[7]=="無"):
                    true_scene += 1

with open("../experiment/"+FILE_NAME+"プロパティ.tsv",mode="r",encoding='utf-8') as obj_file:  
    properties_list = list(csv.reader(obj_file, delimiter='\t'))#開いたファイルをtsv形式で読み込む              
    with open("../experiment/"+FILE_NAME+"SVO結果評価.tsv",mode="w",encoding='utf-8',newline='') as output_file:    #結果の評価をSVO結果評価に出力
        writer = csv.writer(output_file,delimiter="\t")   #出力形式をTSVファイルに設定
        writer.writerow(["","正解数","総数","割合"])
        writer.writerow(["主語",true_subject,all_subject,true_subject/all_subject])
        writer.writerow(["述語",true_hasPredicate,all_hasPredicate,true_hasPredicate/all_subject])
        writer.writerow(["目的語",true_obj,all_obj,true_obj/all_obj])
        writer.writerow(["全単語",true_subject+true_hasPredicate+true_obj,all_subject+all_hasPredicate+all_obj,(true_subject+true_hasPredicate+true_obj)/(all_subject+all_hasPredicate+all_obj)])#主語、述語、目的語の合計
        writer.writerow(["全場面",true_scene,all_scene,true_scene/all_scene])
        for token_property in properties_list:
            if int(token_property[2]):
                writer.writerow([token_property[0],token_property[1],token_property[2],int(token_property[1])/int(token_property[2])]) 
            else:
                writer.writerow([token_property[0],token_property[1],token_property[2],"該当なし"]) 
print("出力完了")

出力完了


転置対策〇

正解の目的語プロパティ〇
正解の述語プロパティ〇

場面ごとのSVO〇部分と全問間違い分ける、目的語の有無（26だと目的語が存在しない場合もOと表示される）〇

S〇V〇O〇→SVO
S〇V〇O無→SVO
S〇V〇O×→SV

解決策
S〇V〇O〇→○○〇
S〇V〇O無→○○無
S〇V〇O×→○○×

一部正解なら△を入れる。〇



インスタンス内関数の階層（インデントは関数内で使われる関数）

・svo_parse（主語・述語・目的語の取得）

　・subject_search

　　・first_search(重複)

　　・last_search（重複）

　・obj_search
　　
　　・first_search(重複)

　　・last_search（重複）
　
 　・hasPredicate_search
  
　　・first_search(重複)

　　・last_search（重複）
  
・svo_print（取得した主語・述語・目的語をファイル形式で出力、目的語プロパティの種類ごとの正解数取得）

　・svo_cheak（出力結果を比較する関数だが……目的語や述語のプロパティ分けが出来ないので目的語や述語の比較はsvo_printの中で行う）